## 규제

## $l_{1}$, $l_{2}$ 규제

In [ ]:
# l2 규제 적용
layer = keras.layers.Dense(100, activation="elu",
                          kernel_initializer = "he_normal",
                          kernel_initializer = keras.regularizers.l2(0.01))

# l1, l2규제 적용
layer = keras.layers.Dense(100, activation="elu",
                          kernel_initializer = "he_normal",
                          kernel_initializer = keras.regularizers.l1_l2(l1=0.01,
                                                                       l2=0.01))

- 일반적으로 네트워크의 모든 은닉층에 동일한 활성화 함수, 동일한 초기화 전략을 사용하거나 모든 층에 동일한 규제를 적용하기 때문에 동일한 매개변수 값을 반복하는 경우가 많음.
- 해결방법
    - **리팩터링**을 통해 반복문을 사용하도록 함.
    - **functools.partial() 함수를 사용하여 기본 매개변수 값을 사용하여 함수 호출을 감쌈**

In [ ]:
from functools import partial

RegularizedDense = partial(keras.layers.Dense,
                          activation = "elu",
                          kernel_initializer = "he_normal",
                          kernel_regularizer = keras.regularizers.l2(0.01))

model = keras.models.Sequential([
    keras.layers.Flatten(input_shape = [28,28])
    RegularizeDense(300),
    RegularizeDense(100),
    RegularizeDense(10, activation = "softmax",
                   kernel_initializer = "glorot_uniform")
])

## 드롭아웃
- 좋은 신경망에서 조차 드롭아웃을 적용하여 정확도를 1~2%로 상승시킬 정도로 심층 신경망에 적합한 규제 기법 중 하나(*모델의 정확도가 95%일 때 2% 상승하는 것은 오차율이 거의 40% 정도 줄어드는 것을 의미*)
- 일반적으로 (출력층을 제외한) 맨 위의 층부터 세번째 층까지 있는 뉴런에만 드롭아웃을 적용

### 드롭아웃이 성능향상에 도움이 되는 이유
1. 더 안정적인 네트워크를 만들어 일반화 성능이 좋아짐
    - 드롭아웃으로 훈련되 뉴런은 이웃한 뉴런에 맞추어 적응될 수 없음. 즉, 가능한 한 자기 자신이 유용해져야 함.
    - 몇 개의 입력 뉴런에만 지나치게 의존할 수 없음. 모든 입력 뉴런에 주의를 기울여야 함.  -> 입력값의 작은 변화에 덜 민감함
2. 만들어지는 모든 신경망을 평균한 앙상블
    - 각 훈련 스텝에서 고유한 네트워크가 생성된다고 생각. $2^{N}$개의 네트워크가 가능(개개의 뉴런이 있을 수도 없을 수도 있기 때문, N은 드롭아웃이 가능한 뉴런 수). 따라서 같은 네트워크가 두 번 선택될 가능성이 거의 없음. 즉, 10,000번의 훈련 스텝을 진행하면 10,000개의 다른 신경망을 (각각 하나의 훈련 샘플을 사용해서) 훈련하게 돔. 이 신경망은 대부분의 가중치를 공유하고 있기 때문에 아주 독립적이진 않지만, 그럼에도 차이가 있음. 


### 알고리즘
- 훈련스텝
    - 매 훈련 스텝에서 각 뉴런(입력뉴런)은 임시적으로 드롭아웃될 확률 $p$를 가짐. 즉 이번 훈련 스텝에는 완전히 무시되지만 다음 스텝에는 활성화될 수 있음. 하이퍼파라미터 $p$를 드롭아웃 비율이라고 하고 보통 10~50% 사이를 지정
    - 순환 신경망에서는 20\~30%에 가깝고, 합성공 신경망에서는 40~50%에 가까움
- 평가
    - 모든 뉴런을 사용

### 보존확률
- $p=50%%$로 하면 테스트하는 동안 하나의 뉴런이 훈련 때보다 (평균적으로) 두 배 많은 입력 뉴런과 연결됨. 따라서 테스트 시 각 뉴런이 훈련한 것보다 거의 두 배 많은 입력 신호를 받기 때문에 잘 작동하지 않을 것.
- 훈련하고 난 뒤에 각 뉴런의 연결 가중치에 **보존확률($1-p$)**를 곱할 필요가 있음
- 또는 훈련하는 동안 각 뉴런의 출력을 보존 확률로 나눌 수도 있음(완전히 같은 방식은 아니지만 잘 작동)
- 모델이 과대적합되었다면 드롭아웃 비율을 늘리고, 반대로 모델이 훈련 세트에 과소적합되었다면 드롭아웃 비율을 낮춤. 층이 클 때는 드롭아웃 비율을 늘리고 작은 층에는 드롭아웃 비율을 낮춤.
- 많은 최신의 신경망 구조는 마지막 은닉층 뒤에만 드롭아웃을 사용. 드롭아웃을 전체에 적용하는 것이 너무 강하다면 시도.
- 드롭아웃은 수렴을 상당히 느리게 만드는 경향이 있지만 적절하게 튜닝하면 훨씬 좋은 모델을 만듦. 따라서 일반적으로 추가적인 시간과 노력을 기울일 가치가 있음

- 드롭아웃은 훈련하는 동안에만 활성화되므로 훈련 손실과 검증 손실을 비교하면 오해를 일으키기 쉬움. 즉, **훈련 시에 모든 뉴런을 사용하지 않았는데도 검증 손실과 비슷하다면, 모든 뉴런을 사용했을 경우에는 훈련세트의 성능이 훨씬 좋을 수도 있음(훈련세트에 과대적합)**. 따라서 (예를 들어 훈련이 끝난 후)드롭아웃을 빼고 훈련 손실을 평가해야 함

- SELU활성화 함수를 기반으로 자기 정규화하는 네트워크를 규제하고 싶다면 **알파 드롭아웃**을 사용. 이 방법은 입력의 평균과 표준편차를 유지하는 드롭아웃의 한 변종.(일반 드롭아웃이 자기 정규화하는 기능을 망가뜨릴 수 있기 때문)

In [ ]:
# 훈련하는 동안 일부 입력을 랜덤하게 버림(0으로 설정)
# 그 다음 남은 입력을 보존확률로 나눔
# 훈련이 끝난 뒤에는 어떤 작업도 하지 않음.
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28,28]),
    keras.layers.Dropout(rate=0.2),
    keras.layers.Dense(300, activation="elu", kernel_initializer = "he_normal"),
    keras.layers.Dropout(rate=0.2),
    keras.layers.Dense(100, activation="elu", kernel_initializer = "he_normal"),
    keras.layers.Dropout(rate=0.2)
    keras.layers.Dense(10,activation="softmax")
])

## 몬테 카를로 드롭아웃
- 드롭아웃을 수학적으로 정의하여, 드롭아웃 네트워크(즉, 모든 가중치 층 이전에 Dropout 층을 포함한 신경망)와 근사 베이즈 추론(approximate Bayesian inference) 사이에 깊은 관련성을 정립
    - 드롭아웃 네트워크를 훈련하는 것이 심층 가우스 과정(deep Gaussian process)이라 불리는 특정 확률 모델에서 근사 베이즈 추론과 수학적으로 동등하다는 것을 보임
- 성능을 크게 향상시킬 수 있는 **몬테 카를로 드롭아웃(Monte Carlo dropout, MC dropout)** 이라 불리는 강력한 기법을 소개.
    - 훈련된 드롭아웃 모델을 재훈련하거나 전혀 수정하지 않고 성능을 크게 향상시킬 수 있음.
- 드롭아웃으로 만든 예측을 평균하면 일반적으로 드롭아웃 없이 예측한 하나의 결과보다 성능이 더 좋고 더 안정적(더 정확한 불확실성 추정을 제공)
    - 단일 모델 보다 예측을 매우 주의 깊게 하게 됨(확신하지 않음)
    - 확률 추정의 표준 분포를 확인해 불확실한 예측을 더 주의 깊게 다룰 수 있게 됨

In [ ]:
# MC dropout 구현
# model(X) : 넘파이 배열이 아닌 텐서를 반환한다는 것만 빼고는 model.predict(x)와 비슷
# training 매개변수를 지원
# 100개의 예측을 만들어 쌓음.

y_probas = np.stack([model(X_test_scaled, training=True),
                    for sample in range(100)])
y_proba = y_probas.mean(axis=0)

- y_probas -> (100, 10000, 10) 크기의 행렬
    - 몇 개의 모델을 사용할지는 하이퍼파라미터. 이 값이 높을 수록 예측과 불확실성 추정이 더 정확해짐. 하지만 시간이 오래 걸리고, 일정 샘플 수가 넘어가면 성능이 크게 향상되지 않음. -> 균형점을 찾는 것이 중요
- 첫번째 차원(axis=0)을 기준으로 평균하면 한 번의 예측을 수행했을 때와 같은 (10000,10) 크기의 배열을 갖게 됨

In [ ]:
# 드롭아웃을 하지 않은 경우
np.round(model.predict(X_test_scaled[:1]),2)

- **만약 모델이 훈련하는 동안 다르게 작동하는 (BatchNormalization 층과 같은) 층을 가지고 있다면** 훈련 모드를 강제로 설정해서는 안됨.
- 대신 Dropout 층을 MCDropout 클래스로 바꾸어줌
    - AlphaDropout도 마찬가지
    - Sequential API에서는 MCDropout클래스를 무조건 사용해야 하지만, Subclassing API혹은 Functional API의 경우 보통의 Dropout 층을 만들고 training=True를 사용하여 호출할 수 있음(Sequential API는 포함된 모든 층의 training 매개변수를 강제로 바꾸기 때문)
- 이미 Dropout을 사용하여 모델을 훈련했다면 Dropout층을 MCDropout으로 바꾸고 기존 모델과 동일한 모델을 새로 만듦. 그 다음 기존 모델의 가중치를 새로운 모델로 복사

In [ ]:
class MCDropout(keras.layers.Dropout):
    def call(self, inputs):
        return super().call(inputs, training=True)

## 맥스-노름 규제

- 맥스-노름 규제는 (배치 정규화를 사용하지 않았을 때) 불안정한 그레디언트 문제를 완화하는 데 도움을 줄 수 있음
- 각각의 뉴런에 대해 입력의 연결 가중치 $w$ 가 $||w||_{2}<=r$이 되도록 제한.
- $r$ : 맥스-노름 하이퍼파라미터
    - 줄이면 규제의 양이 증가하여 과대적합을 감소시키는 데 도움이 됨.
- $||.||$ : $l_{2}$norm

- 매 훈련 스텝이 끝난 후 $||w||_{2}$을 계산하고 이 값이 $r$보다 크다면 $w$의 스케일을 조정. ($w<= w\frac{r}{||w||_{2}}$, 해당 gradient의 norm이 r이 되도록 조정)

In [ ]:
keras.layers.Dense(100, activation = "elu", kernel_initializer="he_normal",
                  kernel_constraint=keras.constraints.max_norm(1.))

# 매 반복이 끝난 후 max norm()이 반환한 객체를 호출하고 
# 스케일이 조정된 가중치를 반환받음
# max_norm() 함수 매개변수 axis의 디폴트 0
# Dense : 각 가중치 벡터에 독립적으로 적용
# 합성공 층에서는 axis=[0,1,2] -> 채널축에 독립적으로 적용

## 요약 및 실용적인 가이드라인
- 기본 DNN 설정
    - 커널 초기화 : He초기화
    - 활성화 함수 : ELU
    - 정규화 : 얕은 신경일 경우 없음. 깊은 신경망이라면 배치 정규화
    - 규제 : 조기종료(필요하면 $l_{2}$ 규제 추가)
    - 옵티마이저 : 모멘텀(or RMSProp이나 Nadam)
    - 학습률 스케줄 : 1사이클
- 자기 정규화를 위한 DNN 설정
    - 커널 초기화 : lecun 초기화
    - 활성화 함수  SELU
    - 정규화 : 없음(자기 정규화)
    - 규제 : 필요하다면 알파 드롭아웃
    - 옵티마이저 : 모멘텀(or RMSProp이나 Nadam)
    - 학습률 스케줄 : 1사이클
- 입력 특성 정규화
- 사전 훈련/비지도 사전훈련/보조 작업을 통한 사전 훈련

#### 그 밖의 예외적인 경우
- 희소모델 필요: $l_{1}$ / TF-MOT(자기 정규화를 깨트리므로 기본 DNN 설정 필요)
- 빠른 모델 : 층 개수를 줄이고 배치 정규화 층을 이전 층에 합침. LeaklyReLU/ReLU와 같은 활성화 함수 사용. 부동소수점 정밀도를 32->16/8비트로 낮춤. TF-MOT
- 위험에 민감하고 예측 속도가 매우 중요하지 않은 애플리케이션이라면 성능을 올리고 불확실성 추정과 신뢰할 수 있는 확률 추정을 얻기 위해 MC 드롭아웃을 사용